In [21]:
from opensky_api import OpenSkyApi
api = OpenSkyApi()
s = api.get_states()
from row64tools import ramdb
import paramiko


In [ ]:
import pandas as pd
from opensky_api import OpenSkyApi
import time
from datetime import datetime, timedelta



def fetch_states_bbox(api, bbox=None, time_secs=0):
    """Fetch aircraft states over bbox at given time (0 = now)."""
    return api.get_states(time_secs=time_secs, bbox=bbox)

def get_flight_route_info(api, callsign, current_time):
    """Try to get origin/destination from recent flights data."""
    try:
        # Look for flights from the past 24 hours
        begin_time = int((current_time - timedelta(days=1)).timestamp())
        end_time = int(current_time.timestamp())
        
        flights = api.get_flights_by_aircraft(callsign.strip(), begin_time, end_time)
        
        if flights and len(flights) > 0:
            # Get the most recent flight
            recent_flight = flights[-1]
            return {
                'origin_airport': getattr(recent_flight, 'estDepartureAirport', None),
                'destination_airport': getattr(recent_flight, 'estArrivalAirport', None)
            }
    except Exception as e:
        print(f"Could not fetch route for {callsign}: {e}")
    
    return {'origin_airport': None, 'destination_airport': None}

def states_to_dataframe(states, include_routes=False, api=None):
    """Convert OpenSkyStates object to pandas DataFrame."""
    if not states or not states.states:
        return pd.DataFrame()
    
    # Category code lookup
    category_lookup = {
        0: "No information at all",
        1: "No ADS-B Emitter Category Information",
        2: "Light (< 15500 lbs)",
        3: "Small (15500 to 75000 lbs)",
        4: "Large (75000 to 300000 lbs)",
        5: "High Vortex Large (aircraft such as B-757)",
        6: "Heavy (> 300000 lbs)",
        7: "High Performance (> 5g acceleration and 400 kts)",
        8: "Rotorcraft",
        9: "Glider / sailplane",
        10: "Lighter-than-air",
        11: "Parachutist / Skydiver",
        12: "Ultralight / hang-glider / paraglider",
        13: "Reserved",
        14: "Unmanned Aerial Vehicle",
        15: "Space / Trans-atmospheric vehicle",
        16: "Surface Vehicle – Emergency Vehicle",
        17: "Surface Vehicle – Service Vehicle",
        18: "Point Obstacle (includes tethered balloons)",
        19: "Cluster Obstacle",
        20: "Line Obstacle"
    }
    
    rows = []
    current_time = datetime.now()
    
    for s in states.states:
        category_code = getattr(s, 'category', None)
        
        # Get route info if requested and callsign is available
        route_info = {'origin_airport': None, 'destination_airport': None}
        if include_routes and api and s.callsign and s.callsign.strip():
            route_info = get_flight_route_info(api, s.callsign, current_time)
        
        rows.append({
            "timestamp": pd.Timestamp.now(),
            "icao24": s.icao24,
            "callsign": s.callsign.strip() if s.callsign else None,
            "origin_country": s.origin_country,
            "time_position": s.time_position,
            "last_contact": s.last_contact,
            "longitude": s.longitude,
            "latitude": s.latitude,
            "baro_altitude": s.baro_altitude,
            "geo_altitude": s.geo_altitude,
            "on_ground": s.on_ground,
            "velocity_m_s": s.velocity,
            "true_track": getattr(s, 'true_track', s.heading if hasattr(s, 'heading') else None),
            "vertical_rate": s.vertical_rate,
            "squawk": s.squawk,
            "spi": s.spi,
            "position_source": s.position_source,
            "category_code": category_code,
            "category": category_lookup.get(category_code, "Unknown Category"),
            "The `origin_airport` field in the code is used to store the estimated departure airport for a particular aircraft. This information is obtained by looking at recent flight data for the aircraft within the past 24 hours. The `get_flight_route_info` function attempts to retrieve the origin airport from the most recent flight data available for the aircraft identified by its callsign.
            origin_airport": route_info['origin_airport'],
            "destination_airport": route_info['destination_airport'],
        })
    
    return pd.DataFrame(rows)

def clean_dataframe_for_export(df):
    """Clean DataFrame by replacing None values and ensuring proper data types."""
    df_clean = df.copy()
    
    # Replace None values with appropriate defaults
    string_columns = ['callsign', 'origin_country', 'category', 'origin_airport', 'destination_airport']
    numeric_columns = ['time_position', 'last_contact', 'longitude', 'latitude', 
                      'baro_altitude', 'geo_altitude', 'velocity_m_s', 'true_track', 
                      'vertical_rate', 'category_code']
    
    # Handle string columns - replace None with empty string
    for col in string_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna('')
    
    # Handle numeric columns - replace None with 0 or NaN
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0)
    
    # Handle boolean columns
    if 'on_ground' in df_clean.columns:
        df_clean['on_ground'] = df_clean['on_ground'].fillna(False)
    if 'spi' in df_clean.columns:
        df_clean['spi'] = df_clean['spi'].fillna(False)
    
    # Handle other columns that might have None
    remaining_cols = [col for col in df_clean.columns if col not in string_columns + numeric_columns + ['on_ground', 'spi', 'timestamp']]
    for col in remaining_cols:
        df_clean[col] = df_clean[col].fillna('')
    
    return df_clean

def main(include_routes=False):
    # Connect to OpenSky
    api = OpenSkyApi()
    
    # Continental US bounding box
    us_bbox = (24.396308, 49.384358, -125.0, -66.93457)
    
    # Fetch data
    states = fetch_states_bbox(api, bbox=us_bbox)
    df = states_to_dataframe(states, include_routes=include_routes, api=api)
    
    print(f"Created DataFrame with {len(df)} records")
    if include_routes:
        print("Note: Route fetching enabled - this may take longer due to API calls")
    return df

# Create the dataframe (set include_routes=True to fetch origin/destination)
aircraft_df = main(include_routes=False)  # Change to True to enable route fetching

# Clean version ready for ramdb export
aircraft_df_clean = clean_dataframe_for_export(aircraft_df)

if __name__ == "__main__":
    print(f"DataFrame columns: {list(aircraft_df.columns)}")
    print(aircraft_df.head())
    print(f"DataFrame shape: {aircraft_df.shape}")
    print(f"Clean DataFrame ready for export: {aircraft_df_clean.shape}")
    if not aircraft_df.empty:
        print(f"\nColumns in DataFrame: {list(aircraft_df.columns)}")
        print(f"\nUnique categories found:")
        if 'category_code' in aircraft_df.columns and 'category' in aircraft_df.columns:
            print(aircraft_df[['category_code', 'category']].value_counts())
        else:
            print("Category columns not found in DataFrame")
    else:
        print("DataFrame is empty - no aircraft data retrieved")


# aircraft_df_clean_5=aircraft_df_clean.iloc[:5]

# Create a file 
ramdb.save_from_df(aircraft_df_clean, "C:\\Users\\mikha\\aircraft_df.ramdb")

localfile = r"C:\Users\mikha\aircraft_df.ramdb"

# remote_path = "/var/www/ramdb/loading/RAMDB.Row64/Temp/opensky.ramdb"
remote_path = "/var/www/ramdb/live/RAMDB.Row64/Stream/opensky.ramdb"

# Ubuntu server credentials
hostname = "192.168.1.23"   # Replace with your Ubuntu server's local IP
port = 22
username = "row64"   # Ubuntu login username
password = "temp7"   # Or use SSH key auth

# Transmit the data
try:
    # Connect via SSH
    transport = paramiko.Transport((hostname, port))
    transport.connect(username=username, password=password)

    # Start SFTP session
    sftp = paramiko.SFTPClient.from_transport(transport)

    # Upload file
    sftp.put(localfile, remote_path)
    print(f"✅ File uploaded to {remote_path} on {hostname}")

    # Close connection
    sftp.close()
    transport.close()

except Exception as e:
    print(f"❌ Failed to upload: {e}")

Created DataFrame with 7481 records
DataFrame columns: ['timestamp', 'icao24', 'callsign', 'origin_country', 'time_position', 'last_contact', 'longitude', 'latitude', 'baro_altitude', 'geo_altitude', 'on_ground', 'velocity_m_s', 'true_track', 'vertical_rate', 'squawk', 'spi', 'position_source', 'category_code', 'category', 'origin_airport', 'destination_airport']
                   timestamp  icao24 callsign origin_country  time_position  \
0 2025-09-15 13:52:49.499619  ab5cb7   N831GR  United States     1757958474   
1 2025-09-15 13:52:49.499681  a5f852   RTY484  United States     1757958765   
2 2025-09-15 13:52:49.499694  ab1644  UAL2675  United States     1757958764   
3 2025-09-15 13:52:49.499699  a53edd   GPD437  United States     1757958764   
4 2025-09-15 13:52:49.499704  ab6fdd  AAL1993  United States     1757958764   

   last_contact  longitude  latitude  baro_altitude  geo_altitude  ...  \
0    1757958474  -115.8227   36.4542        3185.16       3291.84  ...   
1    175795